#   LangGraph 활용 - 메모리 타입

- **Short-term Memory (단기 메모리)**: 단일 대화 세션 내에서 상호작용을 추적
- **Long-term Memory (장기 메모리)**: 세션 간에 사용자별 또는 애플리케이션 수준의 데이터를 저장

![Memory Types](https://langchain-ai.github.io/langgraph/concepts/img/memory/short-vs-long.png)

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

from pprint import pprint
import json

`(3) Langsmith tracing 설정`

In [ ]:
# Langsmith tracing 여부를 확인 
import os
print(os.getenv('LANGSMITH_TRACING'))

---

## **레스토랑 메뉴 DB**


`(1) 문서 로드`

In [ ]:
from langchain.document_loaders import TextLoader
import re

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

In [ ]:
# 와인 메뉴 텍스트를 로드
wine_loader = TextLoader("./data/restaurant_wine.txt", encoding="utf-8")

# 와인 메뉴 문서 생성
wine_docs = wine_loader.load()

# 와인 메뉴 문서 분할
wine_documents = []
for doc in wine_docs:
    wine_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(wine_documents)}개의 와인 메뉴 항목이 처리되었습니다.")
for doc in wine_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

`(2) 벡터스토어 저장`

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# 임베딩 모델 생성
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 메뉴판 Chroma 인덱스 생성
menu_db = Chroma.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model,   
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

# 와인 메뉴 Chroma 인덱스 생성
wine_db = Chroma.from_documents(
    documents=wine_documents, 
    embedding=embeddings_model,   
    collection_name="restaurant_wine",
    persist_directory="./chroma_db",
)

In [ ]:
print(menu_db._collection.count())
print(wine_db._collection.count())

`(3) 벡터 검색기 테스트`

In [ ]:
# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)

# 쿼리 테스트
query = "시그니처 스테이크의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

In [ ]:
wine_retriever = wine_db.as_retriever(
    search_kwargs={'k': 2},
)

query = "스테이크와 어울리는 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

`(4) 레스토랑 메뉴 도구 설정`

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 인덱스 로드 
menu_db = Chroma(
    embedding_function=embeddings_model,   
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

wine_db = Chroma(
    embedding_function=embeddings_model,   
    collection_name="restaurant_wine",
    persist_directory="./chroma_db",
)


# Tool 정의 
@tool
def search_menu(query: str, k: int = 2) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=k)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

@tool
def search_wine(query: str, k: int = 2) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant wine menu information from the encrypted database.
    Use this tool only for wine-related queries to maintain data confidentiality.
    """
    docs = wine_db.similarity_search(query, k=k)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

---

## **단기 메모리 (Short-term Memory)**

- **개념**

    - Short-term memory는 **스레드 범위 메모리**로, 단일 대화 세션 내에서 진행 중인 대화를 추적
    - LangGraph는 이를 에이전트의 **state**의 일부로 관리하며, **checkpointer**를 사용해 데이터베이스에 지속적으로 저장

- **특징**

    - **대화 연속성**: 메시지 기록을 통해 대화 맥락 유지
    - **상태 지속성**: 체크포인트를 통해 언제든지 대화 재개 가능
    - **실시간 업데이트**: 그래프 실행 또는 단계 완료 시 자동 업데이트


---

### 1. **MemorySaver**

- **MemorySaver**는 LangGraph에서 제공하는 스레드 기반의 단기 메모리(short-term memory)

- **단기 메모리**는 하나의 **대화 세션** 동안만 정보를 유지

- LangGraph는 **에이전트의 상태**로서 단기 메모리를 관리하며, 체크포인터를 통해 데이터베이스에 저장됨

- 메모리는 그래프 실행 또는 단계 완료 시 **업데이트**되며, 각 단계 시작 시 상태를 읽어들임

`(1)  상태 정의`

In [ ]:
from typing import Annotated, Optional
from typing_extensions import TypedDict
from operator import add

# 상태 정의
class State(TypedDict):
    query: str
    search_results: Annotated[list[str], add]
    summary: Optional[str]

`(2) 노드 정의`

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import ToolNode
from pydantic import BaseModel, Field
from typing import List


# 구조화 출력 정의
class Sentence(BaseModel):
    """
    Represents a sentence in the search results.
    """
    text: str = Field(..., description="The text of the sentence.")
    source: str = Field(..., description="The source of the sentence.")

class SummaryResult(BaseModel):
    """
    Represents the summary of the search results.
    """
    summaries: List[Sentence] = Field(
        ...,
        description="A list of sentences summarizing the search results."
    )

# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm = ChatOpenAI(model="gpt-4.1-mini")
tools = [search_menu, search_wine]
llm_with_tools = llm.bind_tools(tools)

# 도구 노드 정의 
tool_node = ToolNode(tools=tools)

# Summary 채인
system_prompt = """
You are an AI assistant helping a user find information about a restaurant menu and wine list. 
Answer in the same language as the user's query.
"""

user_prompt = """
Summarize the following search results.

<GUIDELINES>
- Provide a brief summary of the search results.
- Include the key information from the search results.
- Use 1-2 sentences to summarize the information.
- Ensure the summary is concise and relevant to the user's query.
</GUIDELINES>

<Search Results>
{search_results}
</Search Results>

<User Query>
{query} 
</User Query>
"""


summary_prompt = ChatPromptTemplate.from_messages(
    messages=[
        ("system", system_prompt),
        ("user", user_prompt),
    ]
)

summary_chain = summary_prompt | llm.with_structured_output(SummaryResult)

In [ ]:
# 노드 정의 
def search_node(state: State): 
    """Performs a database search based on the query."""
    query = state['query']


    # 검색 도구 사용 
    result = llm_with_tools.invoke(query)
    if result.tool_calls:
        tool_results = tool_node.invoke({"messages": [result]})

    # 도구 메시지 확인
    if tool_results['messages']:
        print(f"검색 문서의 개수 : {len(tool_results['messages'])}")
        return {"search_results": tool_results['messages']}
    
    return {"query": query}

def summarize_node(state: State):
    """Creates a concise summary of the search results."""
    search_results = state.get('search_results', [])
    user_query = state.get('query', '')

    if search_results:
        # 결과가 있을 경우 요약 생성
        summary_result = summary_chain.invoke({"search_results": search_results, "query": user_query})
        summary_sentences = summary_result.summaries

        # 요약 문장들을 문자열로 변환
        summary = " ".join([s.text for s in summary_sentences])

    else:
        summary = "No results found."
        
    return {"summary": summary}

`(3) StateGraph 구성`

In [ ]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# StateGraph 생성
workflow = StateGraph(State)

# 노드 추가
workflow.add_node("search", search_node)
workflow.add_node("summarize", summarize_node)

# 엣지 연결
workflow.add_edge(START, "search")
workflow.add_edge("search", "summarize")
workflow.add_edge("summarize", END)

`(4) 체크포인트 설정`

- 그래프를 컴파일할 때 체크포인터를 지정
- **InMemorySaver**: 디버깅/테스트 용도로 사용

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

# 메모리 저장소 생성
checkpointer = InMemorySaver()

# 메모리 저장소를 지정하여 그래프 컴파일
graph_memory = workflow.compile(checkpointer=checkpointer)

# 그래프 출력
display(Image(graph_memory.get_graph().draw_mermaid_png()))

`(5) 체크포인터 사용`
- 메모리 사용 시 `thread_id`를 지정 
- 체크포인터는 그래프의 각 단계에서 상태를 기록 (그래프 각 단계의 모든 상태를 컬렉션으로 저장)
- 나중에 `thread_id`를 사용하여 이 스레드에 접근 가능 

In [ ]:
# thred_id 설정
config = {"configurable": {"thread_id": "session_123"}}

# 초기 메시지 설정
initial_input = {"query": "스테이크 메뉴가 있나요? 어울리는 와인도 추천해주세요."}

# 그래프 실행
output = graph_memory.invoke(initial_input, config)

In [ ]:
# 최종 결과 출력
pprint(output)

In [ ]:
# thread_id 설정 (다른 스레드에서 실행)
config = {"configurable": {"thread_id": "session_456"}}

# 초기 메시지 설정
initial_input = {"query": "채식주의자를 위한 메뉴가 있나요? 주재료가 무엇인지도 알려주세요."}

# 그래프 실행
output = graph_memory.invoke(initial_input, config)

# 최종 결과 출력
pprint(output)

`(6) 상태 가져오기`

- `graph.get_state(config)`는 스레드의 **최신 상태**를 조회하는 메서드임

- 상태 조회 시 필수적으로 **thread_id**를 지정해야 함

- **checkpoint_id** 지정 시 특정 체크포인트 시점의 상태를 가져올 수 있음

In [ ]:
# 현재 상태 출력 (가장 최근 상태)
config = {"configurable": {"thread_id": "session_123"}}
current_state = graph_memory.get_state(config)

# 현재 상태의 속성 출력
print(f"config: {current_state.config}")
print("-" * 100)
print(f"next: {current_state.next}")
print("-" * 100)
print("values:")
pprint(current_state.values)

`(7) 상태 히스토리 가져오기`

- `graph.get_state_history(config)`로 스레드의 **전체 실행 기록**을 조회함

- 반환값은 **StateSnapshot 객체** 리스트 형태임

- 리스트의 첫 번째 요소가 **가장 최근 체크포인트**를 나타냄

In [ ]:
# 상태 히스토리 출력
config = {"configurable": {"thread_id": "session_123"}}
state_history = list(graph_memory.get_state_history(config))

for i, state_snapshot in enumerate(state_history):
    print(f"  Checkpoint {i}:")
    print(f"    Values: {state_snapshot.values.items()}")
    print(f"    Next: {state_snapshot.next}")
    print(f"    Config: {state_snapshot.config}")
    print("-" * 100)

`(5)  재생 (Replay)`

- `thread_id`와 **checkpoint_id**를 함께 지정하면 특정 체크포인트 이후부터 실행 가능

- 체크포인트 이전 단계는 **재생**(replay)만 하고 실제로 실행하지 않음

- 이는 불필요한 단계 **재실행을 방지**하며 효율적인 처리를 가능하게 함

In [ ]:
# 요약('summarize')이 처리되기 이전 시점의 체크포인트 찾기
snapshot_before_summarize = None
for state_snapshot in state_history:
    if state_snapshot.next == ('summarize',):
        snapshot_before_summarize = state_snapshot
        break

print(f"Config before summarize: {snapshot_before_summarize.config}")
print("-" * 100)
print(f"Next before summarize: {snapshot_before_summarize.next}")
print("-" * 100)
print(f"Values before summarize: {snapshot_before_summarize.values.keys()}")
print("-" * 100)

In [ ]:
snapshot_before_summarize.config

In [ ]:
# 체크포인트 이전의 입력이 필요하지 않기 때문에 빈 입력으로 invoke -> 'search' 노드부터 다시 시작 (재생)
for step in graph_memory.stream(None, snapshot_before_summarize.config, stream_mode="values"):
    print(step.keys())
    print("-" * 100)

In [ ]:
step

In [ ]:
# 상태 히스토리 출력 -> Replay 이후 상태 히스토리 포함 
config = {"configurable": {"thread_id": "session_123"}}
state_history = list(graph_memory.get_state_history(config))

for i, state_snapshot in enumerate(state_history):
    print(f"  Checkpoint {i}:")
    print(f"    Values: {state_snapshot.values.keys()}")
    print(f"    Next: {state_snapshot.next}")
    print(f"    Config: {state_snapshot.config}")
    print(f" Summary: {state_snapshot.values.get('summary', '***')}")
    print("-" * 100)

`(8) 상태 업데이트`

- `graph.update_state(config, values, as_node=None)`로 그래프의 **상태를 직접 수정**함

- **values** : 업데이트할 값을 지정함

- **as_node** : 업데이트를 수행할 노드를 지정 (선택 사항)

In [ ]:
# 요약('summarize')이 처리되기 이전 시점의 체크포인트
snapshot_before_summarize.config

In [ ]:
# 체크포인터에서 'snapshot_before_summarize' 상태의 values 출력
snapshot_before_summarize.values.keys()

In [ ]:
snapshot_before_summarize.values

In [ ]:
# 상태 업데이트 -> 'search' 노드에서 '쿼리'를 수정하고 다시 실행
update_input = {"query": "스테이크 메뉴가 있나요? 메뉴 이름, 가격 정보만 간단하게 출력하세요."}

graph_memory.update_state(
    snapshot_before_summarize.config,   # 재생 시점의 config
    update_input, 
)

# 업데이트된 상태 가져와서 출력 
updated_state = graph_memory.get_state(config)

pprint(updated_state.values)

In [ ]:
# 업데이트된 상태로 이어서 실행 (재생) : thread_id만 전달 필요 (update_state를 실행한 후에는 새로운 체크포인트가 생성되기 때문에)
for step in graph_memory.stream(None, config, stream_mode="values"):
    print(step.keys())
    print("-" * 100)

In [ ]:
# 업데이트된 상태의 히스토리 출력
state_history_after_update = list(graph_memory.get_state_history(config))

for i, state_snapshot in enumerate(state_history_after_update):
    print(f"  Checkpoint {i}:")
    print(f"    Values: {state_snapshot.values.keys()}")
    print(f"    Next: {state_snapshot.next}")
    print(f"    Summary: {state_snapshot.values.get('summary', '***')}")
    print("-" * 100)

In [ ]:
# 최종 상태 출력
final_state = graph_memory.get_state(config)

pprint(final_state.values)

---

### 2. **메시지 관리하기**

- **긴 대화 기록**은 LLM의 컨텍스트 윈도우 제한으로 인한 오류나 성능 저하를 초래함

- 메모리 관리는 **정확성**과 **응답 시간**, **비용** 사이의 균형이 필요함

- 주요 해결책으로 **메시지 목록 편집**과 **과거 대화 요약**이 있음

`(1) 긴 대화 관리 - 메시지 트리밍 (Trimming)`

- **컨텍스트 제한**으로 인해 LLM이 처리할 수 있는 메시지 길이에 제약이 있음
- 효율적인 **토큰 관리**를 통해 비용을 절감하고 시스템 성능을 최적화할 수 있음
- 신속한 **응답 속도**를 위해 메시지 길이와 복잡성 조절이 필수적임

In [ ]:
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import InMemorySaver

from langchain_core.messages.utils import trim_messages, count_tokens_approximately
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langchain_openai import ChatOpenAI

from IPython.display import Image, display

# 메시지 상태 정의
class GraphState(MessagesState):
    ...

# 긴 대화에서 컨텍스트 윈도우 초과를 방지

# LLM 모델에 도구를 바인딩 
llm = ChatOpenAI(model="gpt-4.1-mini")
tools = [search_menu, search_wine]
llm_with_tools = llm.bind_tools(tools=tools)


# 에이전트 실행 노드 
def call_model_with_trimming(state: GraphState):
    system_prompt = SystemMessage("""You are a helpful AI assistant. Please respond to the user's query to the best of your ability!

중요: 답변을 제공할 때 반드시 정보의 출처를 명시해야 합니다. 출처는 다음과 같이 표시하세요:
- 도구를 사용하여 얻은 정보: [도구: 도구이름]
- 모델의 일반 지식에 기반한 정보: [일반 지식]

항상 정확하고 관련성 있는 정보를 제공하되, 확실하지 않은 경우 그 사실을 명시하세요. 출처를 명확히 표시함으로써 사용자가 정보의 신뢰성을 판단할 수 있도록 해주세요.""")
    

    # 최근 메시지만 유지 (메시지 수 기준)
    trimmed_messages = trim_messages(
        state["messages"],
        strategy="last",  # 마지막 메시지부터 유지
        token_counter=len,  # 메시지 개수로 계산   (count_tokens_approximately는 토큰 수를 계산하는 데 사용되지만, 여기서는 메시지 개수로 제한)
        max_tokens=5,  # 최대 5개의 메시지 유지
        start_on="human",  # 사람 메시지로 시작
        end_on=("human", "tool"),  # 사람 또는 도구 메시지로 종료
        include_system=True,  # 시스템 메시지 포함
    )
    # 트리밍된 메시지 개수와 토큰 수 출력
    print(f"트리밍된 메시지 개수: {len(trimmed_messages)}")
    print(f"트리밍된 토큰 수: {count_tokens_approximately(trimmed_messages)}")

    # 시스템 메시지와 이전 메시지를 결합하여 모델 호출
    messages = [system_prompt] + trimmed_messages

    response = llm_with_tools.invoke(messages)

    # 메시지 리스트로 반환하고 상태 업데이트 
    return {
        "messages": [response]
    }

In [ ]:
# 그래프 구성
builder = StateGraph(GraphState)

builder.add_node("agent", call_model_with_trimming) 
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "agent")

# tools_condition을 사용한 조건부 엣지 추가
builder.add_conditional_edges(
    "agent",
    tools_condition,
)

builder.add_edge("tools", "agent")

# 메모리 저장소 생성
checkpointer = InMemorySaver()

# 메모리 저장소를 지정하여 그래프 컴파일
graph_memory_trimmer = builder.compile(checkpointer=checkpointer)

# 그래프 출력
display(Image(graph_memory_trimmer.get_graph().draw_mermaid_png()))

In [ ]:
# thred_id 설정
config = {"configurable": {"thread_id": "session_123"}}

# 초기 메시지 설정
messages = [HumanMessage(content="스테이크 메뉴의 가격은 얼마인가요")]

# 그래프 실행
for step in graph_memory_trimmer.stream({"messages": messages}, config, stream_mode="values"):
    if "messages" in step:
        step["messages"][-1].pretty_print()
        print("-" * 100)

In [ ]:
# thred_id 설정 유지한 상태에서 다른 메시지로 그래프 실행
config = {"configurable": {"thread_id": "session_123"}}
messages = [HumanMessage(content="둘 중에 더 저렴한 메뉴는 무엇인가요?")]

# 그래프 실행 및 결과 출력 
for step in graph_memory_trimmer.stream({"messages": messages}, config, stream_mode="values"):
    if "messages" in step:
        step["messages"][-1].pretty_print()
        print("-" * 100)

In [ ]:
# 전체 메시지 출력
print("전체 메시지 개수: ", len(step['messages']))
for m in step['messages']:
    m.pretty_print()
    print("-" * 100)

In [ ]:
# thred_id 설정 유지한 상태에서 다른 메시지로 그래프 실행
config = {"configurable": {"thread_id": "session_123"}}
messages = [HumanMessage(content="이 메뉴와 곁들이면 좋은 다른 메뉴가 있나요?")]

# 그래프 실행 및 결과 출력
for step in graph_memory_trimmer.stream({"messages": messages}, config, stream_mode="values"):
    if "messages" in step:
        step["messages"][-1].pretty_print()
        print("-" * 100)

In [ ]:
# 전체 메시지 출력
print("전체 메시지 개수: ", len(step['messages']))
for m in step['messages']:
    m.pretty_print()
    print("-" * 100)

`(2) 긴 대화 관리 - 메시지 요약 (Summarization)`

- **정보 손실을 최소화**하면서 컨텍스트 관리
- 오래된 메시지를 요약하여 **메시지 길이**를 줄이고, 해당 메시지를 삭제
- **LangGraph**의 `MessagesState`는 `RemoveMessage` 기능을 활용하여 메시지를 제거 (삭제할 메시지의 ID를 지정하는 "remove" 객체 목록을 반환함)


In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import InMemorySaver

from langchain_core.messages.utils import trim_messages, count_tokens_approximately
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage, RemoveMessage
from langchain_openai import ChatOpenAI

from IPython.display import Image, display
from typing import Any

# 요약 기능이 포함된 확장된 상태 정의
class SummaryState(MessagesState):
    summary: str   # 대화 요약을 저장할 필드

# LLM 모델에 도구를 바인딩 
llm = ChatOpenAI(model="gpt-4.1-mini")
tools = [search_menu, search_wine]  
llm_with_tools = llm.bind_tools(tools=tools)

def summarize_conversation(state: SummaryState):
    """대화 요약 생성 함수"""
    summary = state.get("summary", "")
    messages_to_summarize = state["messages"][:-2]  # 마지막 2개 메시지를 제외한 나머지
    
    if not messages_to_summarize:
        return {"summary": summary}
    
    if summary:
        summary_message = (
            f"지금까지의 대화 요약: {summary}\n\n"
            "다음 새로운 메시지들을 고려하여 요약을 확장하세요:\n"
            f"{format_messages_for_summary(messages_to_summarize)}"
        )
    else:
        summary_message = (
            "다음 대화의 요약을 생성하세요:\n"
            f"{format_messages_for_summary(messages_to_summarize)}"
        )

    # 요약 생성을 위한 메시지 구성
    summary_prompt = [
        SystemMessage(content="당신은 대화 내용을 간결하고 정확하게 요약하는 전문가입니다. 중요한 정보와 결정사항, 컨텍스트를 포함하여 요약해주세요."),
        HumanMessage(content=summary_message)
    ]
    
    response = llm.invoke(summary_prompt)
    
    return {"summary": response.content}

def format_messages_for_summary(messages):
    """메시지들을 요약용 텍스트로 포매팅"""
    formatted = []
    for msg in messages:
        if hasattr(msg, 'type'):
            msg_type = msg.type.upper()
            content = msg.content if hasattr(msg, 'content') else str(msg)
            formatted.append(f"{msg_type}: {content}")
    return "\n".join(formatted)

# 에이전트 실행 노드 (메시지 삭제 및 요약 포함)
def call_model_with_message_management(state: SummaryState):
    system_prompt = SystemMessage("""You are a helpful AI assistant. Please respond to the user's query to the best of your ability!

중요: 답변을 제공할 때 반드시 정보의 출처를 명시해야 합니다. 출처는 다음과 같이 표시하세요:
- 도구를 사용하여 얻은 정보: [도구: 도구이름]
- 모델의 일반 지식에 기반한 정보: [일반 지식]

항상 정확하고 관련성 있는 정보를 제공하되, 확실하지 않은 경우 그 사실을 명시하세요. 출처를 명확히 표시함으로써 사용자가 정보의 신뢰성을 판단할 수 있도록 해주세요.""")
    
    messages = state["messages"]
    summary = state.get("summary", "")
    
    # 메시지가 5개를 초과하면 요약 및 삭제 진행
    if len(messages) > 5:
        print(f"메시지 개수가 {len(messages)}개이므로 요약 및 삭제를 진행합니다.")
        
        # 1. 먼저 삭제될 메시지들로 요약 생성
        summary_result = summarize_conversation(state)
        updated_summary = summary_result["summary"]
        
        # 2. 마지막 2개를 제외한 메시지들을 삭제
        delete_messages = [RemoveMessage(id=m.id) for m in messages[:-2]]
        
        # 3. 남은 메시지들 (최근 2개)
        remaining_messages = messages[-2:]
        
        print(f"삭제할 메시지 개수: {len(delete_messages)}")
        print(f"남은 메시지 개수: {len(remaining_messages)}")
        print(f"업데이트된 요약: {updated_summary[:100]}...")
        
        # 4. 요약이 있으면 시스템 프롬프트에 포함
        if updated_summary:
            enhanced_system_prompt = SystemMessage(
                content=system_prompt.content + f"\n\n이전 대화 요약: {updated_summary}"
            )
        else:
            enhanced_system_prompt = system_prompt
        
        # 5. 모델 호출용 메시지 구성
        model_messages = [enhanced_system_prompt] + remaining_messages
        
    else:
        # 메시지가 적으면 그대로 사용
        delete_messages = []
        model_messages = [system_prompt] + messages
        updated_summary = summary
        print(f"메시지 개수가 {len(messages)}개이므로 삭제하지 않습니다.")

    # 모델 호출
    response = llm_with_tools.invoke(model_messages)

    # 반환할 업데이트 구성
    update = {
        "messages": delete_messages + [response],  # 삭제할 메시지들 + 새 응답
        "summary": updated_summary
    }
    
    return update


# 도구 노드 생성
tool_node = ToolNode(tools)

# 그래프 생성
workflow = StateGraph(SummaryState)

# 노드 추가
workflow.add_node("agent", call_model_with_message_management)
workflow.add_node("tools", tool_node)

# 엣지 추가
workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    tools_condition,
    # {
    #     "tools": "tools",
    #     "__end__": END,
    # }
)
workflow.add_edge("tools", "agent")

# 메모리 세이버와 함께 컴파일
checkpointer = InMemorySaver()
graph_with_summary = workflow.compile(checkpointer=checkpointer)

# 그래프 출력
display(Image(graph_with_summary.get_graph().draw_mermaid_png()))


In [ ]:
# thred_id 설정
config = {"configurable": {"thread_id": "session_123"}}

# 초기 메시지 설정
messages = [HumanMessage(content="스테이크 메뉴의 가격은 얼마인가요")]

# 그래프 실행
for step in graph_with_summary.stream({"messages": messages}, config, stream_mode="values"):
    if "messages" in step:
        step["messages"][-1].pretty_print()
        print("-" * 100)

In [ ]:
# thred_id 설정 유지한 상태에서 다른 메시지로 그래프 실행
config = {"configurable": {"thread_id": "session_123"}}
messages = [HumanMessage(content="둘 중에 더 저렴한 메뉴는 무엇인가요?")]

# 그래프 실행 및 결과 출력 
for step in graph_with_summary.stream({"messages": messages}, config, stream_mode="values"):
    if "messages" in step:
        step["messages"][-1].pretty_print()
        print("-" * 100)

In [ ]:
# 전체 메시지 출력
print("전체 메시지 개수: ", len(step['messages']))
for m in step['messages']:
    m.pretty_print()
    print("-" * 100)

In [ ]:
# thred_id 설정 유지한 상태에서 다른 메시지로 그래프 실행
config = {"configurable": {"thread_id": "session_123"}}
messages = [HumanMessage(content="이 메뉴와 곁들이면 좋은 다른 메뉴가 있나요?")]

# 그래프 실행 및 결과 출력
for step in graph_with_summary.stream({"messages": messages}, config, stream_mode="values"):
    if "messages" in step:
        step["messages"][-1].pretty_print()
        print("-" * 100)

In [ ]:
# 전체 메시지 출력
print("전체 메시지 개수: ", len(step['messages']))
for m in step['messages']:
    m.pretty_print()
    print("-" * 100)